In [ ]:
!pip install nltk
!pip install torch

In [ ]:
import torch
import torch.nn as nn


In [ ]:
''' 
Preprocess Data
'''

import re
import nltk
import string
import fasttext
from keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize

OTHER_TAG = "other"
PUNCT_TAG = "punct"

with open('./utils/tags.txt', encoding="utf-8", errors='ignore') as f:
    tags = set([str.rstrip(tag) for tag in f.readlines()])

def remove_labels(text):
    return re.sub(r'\<\/?[\w-]*\>\s*', "", text).strip()

def tag_token(token, tag):
    if token in string.punctuation:
        return (token, PUNCT_TAG)
    return (token, tag)

def get_tagged_tokens(groups):
    tagged_tokens = []
    for group in groups:
        ref, tag = group[0], group[1]
        if tag not in tags:
            tag = OTHER_TAG
        unlabelled = remove_labels(ref)
        tokens = word_tokenize(unlabelled)
        tagged_tokens.extend(list(map(lambda token: tag_token(token, tag), tokens)))
    return tagged_tokens

''' Attach tags to each token '''
def attach_tags(dataset_path):
    dataset = []
    with open(dataset_path, encoding="utf-8", errors='ignore') as f:
        refs = f.readlines()
        for ref in refs:
            groups = re.findall(r'(\<(.*)\>.*\<\/\2\>)', ref) # format (<tag>...</tag>, tag)
            tagged_tokens = get_tagged_tokens(groups)
            dataset.append(tagged_tokens)
    return dataset

''' Removes labels and tokenizes '''
def tokenized_dataset(dataset_path, sep=" "):
    dataset = []
    with open(dataset_path, encoding="utf-8", errors='ignore') as f:
        refs = f.readlines()
        for ref in refs:
            ref = remove_labels(ref) 
            tokenized = " ".join(word_tokenize(ref))
            dataset.append(tokenized)
    return dataset

def train_word_embedding_model(dataset_path, embedding_dim, retrain=True):
    embedding_dataset_path = './dataset/word_embedding_dataset'
    model_path = './models/word_embedding.bin'

    if retrain:
        with open(embedding_dataset_path, 'w', errors='ignore') as f:
            # fasttext tokenizes by whitespaces
            word_embedding_dataset = tokenized_dataset(dataset_path, sep=" ") 
            f.write("\n".join(word_embedding_dataset))
        model = fasttext.train_unsupervised(embedding_dataset_path, dim=embedding_dim)
        model.save_model(model_path)
        return model
    else:
        return fasttext.load_model(model_path)

def map_to_index(keys, idx_start=0):
    idx_map, idx = {}, idx_start
    for key in keys:
        idx_map[key] = idx
        idx += 1
    return idx_map

dataset_path = './dataset/standardized_dataset.txt'
dataset = attach_tags(dataset_path)
EMBEDDING_DIM = 100
word_embedding_model = train_word_embedding_model(dataset_path, embedding_dim = EMBEDDING_DIM, retrain=True)

all_tags = tags 
all_tags.add(OTHER_TAG)
all_tags.add(PUNCT_TAG)
tag_to_idx = map_to_index(all_tags)

X, y = [], []
for ref in dataset:
    X_ref, y_ref = [], []
    for token, tag in ref:
        X_ref.append(word_embedding_model.get_word_vector(token))
        y_ref.append(tag_to_idx[tag])
    X.append(X_ref)
    y.append(y_ref)

max_length = max(map(lambda ref: len(ref), X))
X = pad_sequences(X, maxlen=max_length, padding='post', truncating='pre', value=float(len(all_tags)), dtype='float32')
y = pad_sequences(y, maxlen=max_length, padding='post', truncating='pre', value=float(len(all_tags)), dtype='float32')


In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
    
        self.lstm = nn.LSTM(
            input_size = self.input_size,
            hidden_size = self.hidden_size,
            num_layers = self.num_layers,
            batch_first = True
        )

    def forward(self, x):
        # Initialise hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        # Initialise internal state
        c0 = torch.zeros(self.num_layers,x.size(0), self.hidden_size)

        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h0, c0))

        return output, (hn, cn)


In [ ]:
'''
Hyperparameters
'''
num_epochs = 1000
learning_rate = 0.001

input_size = EMBEDDING_DIM # Number of features
hidden_size = 100 # Number of features in the hidden state
num_layers = 1 # Number of stacked LSTM layers

output_size = len(all_tags) # Number of output classes

model = Net(input_size, hidden_size, output_size, num_layers)

In [ ]:
'''
Loss Function and Optimiser
'''
criterion = nn.CrossEntropyLoss(ignore_index=15)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# TODO: Obtain data x_train and y_train
import numpy as np
from torch.nn.utils.rnn import pad_packed_sequence
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = torch.tensor(X_train), torch.tensor(X_test)
y_train, y_test = torch.tensor(y_train), torch.tensor(y_test)

def categorical_accuracy(outputs, y, pad_index):
    max_outputs = outputs.argmax(dim = 1, keepdim=True)
    non_padded_elements = (y != pad_index).nonzero()
    correct = max_outputs[non_padded_elements].squeeze(1).eq(y[non_padded_elements])
    return correct.sum() / torch.FloatTensor([y[non_padded_elements].shape[0]])

def train(X_train, y_train):
  for epoch in range(num_epochs):
      outputs = model.forward(X_train)

      optimizer.zero_grad()

      outputs = outputs.view(-1, outputs.shape[-1]) # [batch_size * seq_len, output_dim]
      y_train = y_train.view(-1) # [batch_size * seq_len]

      # Get the loss function
      loss = criterion(outputs, y_train.long())

      # Calculate loss
      loss.backward()

      # Backpropagation
      optimizer.step()

      # Print loss at every 100th epoch
      if epoch % 100 == 0:
          print("Epoch: %d, loss: %1.5f, acc: %1.5f" \
                % (epoch, loss.item(), categorical_accuracy(outputs, y_train, 15)))

def test(X_test, y_test):
    with torch.no_grad():
        outputs = model.forward(X_test)
        outputs.view(-1, outputs.shape[-1])
        y_test = y_test.view(-1)

        loss = criterion(outputs, y_test.long())
        print("Testing loss: %1.5f, f1 macro: %1.5f" \
               % (loss.item(), categorical_accuracy(outputs, y_test, 15)))

train(X_train, y_train)
test(X_test, y_test)